In [1]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

**EBNF:**  
``sequence = aSequence bSequence cSequence``  
``aSequence = "a" {"a"}``  
``bSequence = "b" {"b"}``  
``cSequence = "c" {"c"}``

In [11]:
class Parser1 :
    def __init__(self, source) :
        self.source = source + '$'
        self.sourcePos = 0
    
    def error(self,message) :
        print('Error at position', self.sourcePos)
        print(message)
        assert False
    
    def scan(self) :
        self.sourcePos = self.sourcePos + 1
        self.lookahead = self.source[self.sourcePos]
        
    def expect(self,symbolSet) : 
        if self.lookahead in symbolSet :
            if self.lookahead != '$' : self.scan()
        else :
            self.error(str(symbolSet) + ' expected, found ' + str(self.lookahead))
        
    # sequence = aSequence bSequence cSequence
    def sequence(self) :
        print("sequence",self.sourcePos,self.lookahead)
        self.aSequence()
        self.bSequence()
        self.cSequence()

    # aSequence = "a" {"a"}
    def aSequence(self) :
        print("aSequence",self.sourcePos,self.lookahead)
        self.expect({'a'})
        while self.lookahead in {'a'} :
            self.scan()

    # bSequence = "b" {"b"}
    def bSequence(self) :
        print("bSequence",self.sourcePos,self.lookahead)
        self.expect({'b'})
        while self.lookahead in {'b'} :
            self.scan()
            
    # cSequence = "c" {"c"}
    def cSequence(self) :
        print("cSequence",self.sourcePos,self.lookahead)
        self.expect({'c'})
        while self.lookahead in {'c'} :
            self.scan()


    def parse(self) :
        try:
            # Prologue: Scan first symbol before descent
            self.sourcePos = 0
            self.lookahead = self.source[self.sourcePos]
            # Call start symbol
            self.sequence()
            # Epilogue: Check for EOF
            self.expect({'$'})
            print('Success!')
        except AssertionError :
            print("Aborted!")


In [12]:
p = Parser1("aaaaabcc")
p.parse()

sequence 0 a
aSequence 0 a
bSequence 5 b
cSequence 6 c
Success!


**$a^n b^n c^n$ with attributed EBNF**

  
``sequence (synth match) = aSequence bSequence cSequence  
                          (* sequence.match = (aSequence.count == bSequence.count == cSequence.count *)``  
  
``aSequence (synth count) = "a" (* aSequence.count = 1 *) {"a" (* aSequence.count++ *)}``  
``bSequence (synth count) = "b" (* bSequence.count = 1 *) {"b" (* bSequence.count++ *)}``  
``cSequence (synth count) = "c" (* cSequence.count = 1 *) {"c" (* cSequence.count++ *)}``


In [4]:
class Parser2 :
    def __init__(self, source) :
        self.source = source + '$'
        self.sourcePos = 0
    
    def error(self,message) :
        print('Error at position', self.sourcePos)
        print(message)
        assert False
    
    def scan(self) :
        self.sourcePos = self.sourcePos + 1
        self.lookahead = self.source[self.sourcePos]
        
    def expect(self,symbolSet) : 
        if self.lookahead in symbolSet :
            if self.lookahead != '$' : self.scan()
        else :
            self.error(str(symbolSet) + ' expected, found ' + str(self.lookahead))
        
    # sequence = aSequence bSequence cSequence
    def sequence(self) :
        print("sequence",self.sourcePos,self.lookahead)
        na = self.aSequence()
        nb = self.bSequence()
        nc = self.cSequence()
        return na == nb and  nb == nc

    # aSequence = "a" {"a"}
    def aSequence(self) :
        print("aSequence",self.sourcePos,self.lookahead)
        self.expect({'a'})
        count = 1
        while self.lookahead in {'a'} :
            count += 1
            self.scan()
        return count

    # bSequence = "b" {"b"}
    def bSequence(self) :
        print("bSequence",self.sourcePos,self.lookahead)
        self.expect({'b'})
        count = 1
        while self.lookahead in {'b'} :
            count += 1
            self.scan()
        return count
            
    # cSequence = "c" {"c"}
    def cSequence(self) :
        print("cSequence",self.sourcePos,self.lookahead)
        self.expect({'c'})
        count = 1        
        while self.lookahead in {'c'} :
            count += 1
            self.scan()
        return count


    def parse(self) :
        try:
            # Prologue: Scan first symbol before descent
            self.sourcePos = 0
            self.lookahead = self.source[self.sourcePos]
            # Call start symbol
            result = self.sequence()
            # Epilogue: Check for EOF
            self.expect({'$'})
            if result :
                print('Success!')
            else :
                self.error("Numbers don't match")
        except AssertionError :
            print("Aborted!")


In [9]:
p = Parser2("aaabbbccc")
p.parse()

sequence 0 a
aSequence 0 a
bSequence 3 b
cSequence 6 c
Success!
